In [1]:
from termcolor import colored
import numpy as np

In [2]:
#revisa si el vector es cero
#vector: lista de enteros
def zero_vector(vector):
    if type(vector) is Integer:
        return vector == 0
    else:
        for i in range(0,len(vector)):
            if vector[i] != 0:
                return False
    return True

In [3]:
#revisa si la matriz es cero
#matrix: lista de listas de enteros
def zero_matrix(matrix):
    for i in matrix:
        if not zero_vector(i):
            return False
    return True

In [4]:
#imprime la matriz de una forma mas clara
#matriz: lista de listas
def pm(matriz):
    for i in matriz:
        print(i)

In [5]:
#transpone la matriz
#matriz: lista de listas
def transpose_matrix(matriz):
    npm = np.array
    zipped_rows = zip(*matriz)
    return [list(row) for row in zipped_rows]


In [6]:
#calcula el omega de una hipercolumna en una matriz
#matriz: lista de listas de enteros
#vector: entero, el indice del vector
#dim: 0 o 1, 0 para filas y 1 para columnas
def omeg(matriz, vector, dim):
    counter = 0
    if dim == 0:
        matriz = matriz[vector:] + matriz
        for i in range(len(matriz)):
            if(zero_vector(matriz[i+1])):
                counter += 1
            else:
                break
    if dim == 1:
        t_matriz = transpose_matrix(matriz)
        t_matriz = t_matriz[vector:] + t_matriz
        for i in range(len(t_matriz)):
            if(zero_vector(t_matriz[i+1])):
                counter += 1
            else:
                break
    return counter

In [7]:
#calcula la distancia aparente fuerte de una hipercolumna
#hm: lista de enteros, la hipercolumna
def sdist_Hm(hm):
    om = [0]
    indices = []
    for i in range(0,len(hm)):
        if hm[i] == 0:
            indices.append(i)
    for i in indices:
        om.append(omeg(hm,i,0)+2)
    return max(om) if om else 0

In [8]:
#calcula la distancia aparente fuerte de una matriz
#mat: lista de listas de enteros
def sad(matr):
    mat = matr
    ind_x = []
    ind_y = []
    
    omega_x = []
    epsi_x = 0
    om_x = 0
    omega_Hm_x = []
    omega_y = []
    epsi_y = 0
    om_y = 0
    omega_Hm_y = []
    
    dist_x = 0
    dist_y = 0
    
    #separar filas y columnas
    rows = list([x for x in mat])
    columns = list([x for x in transpose_matrix(mat)])
    
    #distancia aparente respecto a x
    #revisar cuales son las filas no nulas para sacar su omega y distancia aparente
    for i in range(0, len(rows)):
        if not zero_vector(rows[i]):
            ind_x.append(i)
    for i in ind_x:
        omega_x.append(omeg(mat,i,0))
        omega_Hm_x.append(sdist_Hm(rows[i]))
    om_x = max(omega_x)
    epsi_x = max(omega_Hm_x)
    #calcular la distancia aparente respecto a x
    dist_x = epsi_x*(om_x+1)
            
    #distancia aparente respecto a y     
    #revisar cuales son las columnas no nulas para sacar su omega y distancia aparente
    for i in range(0, len(columns)):
        if not zero_vector(columns[i]):
            ind_y.append(i)
    for i in ind_y:
        omega_y.append(omeg(mat,i,1))
        omega_Hm_y.append(sdist_Hm(columns[i]))
    om_y = max(omega_y)
    epsi_y = max(omega_Hm_y)
    #calcular la distancia aparente respecto a y
    dist_y = epsi_y*(om_y+1)
    
    dist = max([dist_x,dist_y])
    
    #recopilar las parejas involucradas
    ip = []
    for i in range(len(omega_Hm_x)):
        if dist == (om_x+1)*omega_Hm_x[i]:
            ip.append([[1,ind_x[i]],omega_Hm_x[i]])
    for i in range(len(omega_Hm_y)):
        if dist == (om_y+1)*omega_Hm_y[i]:
            ip.append([[2,ind_y[i]],omega_Hm_y[i]])
            
    #retornar la distancia aparente fuerte y la lista de parejas involucradas 
    #con sus respectivas distancias aparentes fuertes
    return [dist,ip]

In [9]:
#calcula las duplas que hacen parte de la q-orbita
#q: entero, numero primo de la q-orbita
#a: dupla de enteros, los enteros a0 y a1 de la q-orbita que se quiere calcular
#r: dupla de enteros, r1 y r2 en I = Zr1 x Zr2
def q_orbits(q,a,r):
    parejas = []
    for i in range(r[0]*r[1]):
        coord1 = (a[0] * q**i) % r[0]
        coord2 = (a[1] * q**i) % r[1]
        if [coord1,coord2] not in parejas:
            parejas.append([coord1,coord2])
    return parejas

In [10]:
#calcula las duplas que hacen parte de la q-orbita
#q: entero, numero primo de la q-orbita
#a: lista de duplas de enteros, las duplas de las q-orbitas
#r: dupla de enteros, r1 y r2 en I = Zr1 x Zr2
def generate_matrix(q,a,r):
    coordinates = []
    for j in a:
        qorbits = q_orbits(q,j,r)
        for o in qorbits:
            if o not in coordinates:
                coordinates.append(o) 
    matrix = [[0 for x in range(r[1])] for y in range(r[0])]
    for i in range(r[0]):
        for j in range(r[1]): 
            matrix[i][j] = 0 if [i,j] in coordinates else 1
    return matrix

#conjuga la matriz
def conjugate_matrix(matriz):
    m = matriz
    for i in m:
        for j in range(len(i)):
            i[j] = 1 if (i[j]==0) else 0
    return m

In [11]:
#calcula el soporte de una hipercolumna
#matriz, lista de lista de enteros
#kb, dupla de enteros
def support(matriz, kb):
    indexes = []
    if kb[0] == 1:
        for i in range(len(matriz[kb[1]])):
            if matriz[kb[1]][i] != 0:
                indexes.append([kb[1],i])
    if kb[0] == 2:
        t_matriz = transpose_matrix(matriz)
        for i in range(len(t_matriz[kb[1]])):
            if t_matriz[kb[1]][i] != 0:
                indexes.append([kb[1],i])
    return indexes

In [12]:
# mat: lista de listas, la matriz a la que se le va a calcular la DAM
# q: entero, numero primo de las q-orbitas
# r: dupla de enteros, r1 y r2 en I = Zr1 x Zr2
# d es la distancia aparente fuerte de mat calculada anteriormente
def min_sad(matr,q,r,d,itera):
    iteracion = itera + 1
    mat = matr
    [dista, ip] = sad(mat)
    print(dista, d)
    dist = min(dista,d)
    print(dist)
    S = []
    for pair in ip:
        if pair[1] == 1:
            return dist
        S.append(support(mat,pair[0]))
    S = [item for sublist in S for item in sublist]
    orbits = []
    for s in S:
        orbits.append(q_orbits(q,s,r))
    orbits = [item for sublist in orbits for item in sublist]
    new_mat = mat
    for i in range(len(new_mat)):
        for j in range(len(new_mat[i])):
            new_mat[i][j] = 0 if [i,j] in orbits else new_mat[i][j]
    pm(new_mat)
    if zero_matrix(new_mat):
        print("Distancia aparente fuerte minima: ", dist)
        return
    else:
        min_sad(new_mat,q,r,dist,iteracion)

<b>Ejemplo 1</b>

In [13]:
matrix =  generate_matrix(2,[[0,0],[0,1],[0,3],[1,0],[1,1]],[5,15])
pm(matrix)

[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1]
[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1]
[0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [14]:
distancia = sad(matrix)[0]
distancia

6

In [15]:
min_sad(matrix,2,[5,15],distancia,0)

6 6
6
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1]
[0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1]
[0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1]
16 6
6
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1]
[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1]
[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1]
[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1]
12 6
6
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Distancia aparente fuerte minima:  6


<b>Ejemplo 2</b>

In [16]:
matrix = generate_matrix(2,[[0,0],[0,3],[0,5],[0,7],[1,0],[1,2],[1,4]],[5,15])
pm(matrix)

[0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1]


In [17]:
distancia = sad(matrix)[0]
distancia

8

In [18]:
min_sad(matrix,2,[5,15],distancia,0)

8 8
8
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1]
8 8
8
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Distancia aparente fuerte minima:  8


<b>Ejemplo 3</b>

In [32]:
%%time
matrix = generate_matrix(2,[[0,3],[0,5],[1,1]],[5,15])
pm(matrix)
print("")
distancia = sad(matrix)[0]
print("Distancia inicial:",distancia)
min_sad(matrix,2,[5,15],distancia,0)
print("")

[1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1]
[1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

Distancia inicial: 3
3 3
3
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
4 3
3
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Distancia aparente fuerte minima:  3

CPU times: user 0 ns, sys: 47 ms, total: 47 ms
Wall time: 26.1 ms


<b>Ejemplo 4</b>

In [20]:
matrix = conjugate_matrix(generate_matrix(2,[[0,0],[1,0],[1,3]],[5,7]))
pm(matrix)

[1, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 1, 0, 1, 1]
[1, 0, 0, 1, 0, 1, 1]
[1, 0, 0, 1, 0, 1, 1]
[1, 0, 0, 1, 0, 1, 1]


In [21]:
distancia = sad(matrix)[0]
distancia

7

In [22]:
min_sad(matrix,2,[5,7],distancia,0)

7 7
7
[0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 1, 0, 1, 1]
[1, 0, 0, 1, 0, 1, 1]
[1, 0, 0, 1, 0, 1, 1]
[1, 0, 0, 1, 0, 1, 1]
6 7
6
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
Distancia aparente fuerte minima:  6


<b>Ejemplo 5</b>

In [23]:
matrix = conjugate_matrix(generate_matrix(2,[[1,0],[1,3]],[5,7]))
pm(matrix)

[0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 1, 0, 1, 1]
[1, 0, 0, 1, 0, 1, 1]
[1, 0, 0, 1, 0, 1, 1]
[1, 0, 0, 1, 0, 1, 1]


In [24]:
distancia = sad(matrix)[0]
distancia

6

In [25]:
distancia = sad(matrix)[0]
min_sad(matrix,2,[5,7],distancia,0)

6 6
6
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
Distancia aparente fuerte minima:  6


<b>Ejemplo 6</b>

In [26]:
matrix = conjugate_matrix(generate_matrix(2,[[1,0],[2,5],[2,7]],[3,15]))
pm(matrix)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1]
[1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0]


In [27]:
distancia = sad(matrix)[0]
distancia

20

In [28]:
min_sad(matrix,2,[3,15],distancia,0)

20 20
20
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Distancia aparente fuerte minima:  20


<b>Ejemplo 7 (Diana)</b>

In [29]:
matrix = generate_matrix(2,[[0,0],[1,0],[0,3]],[5,7])
pm(matrix)

[0, 1, 1, 0, 1, 0, 0]
[0, 1, 1, 1, 1, 1, 1]
[0, 1, 1, 1, 1, 1, 1]
[0, 1, 1, 1, 1, 1, 1]
[0, 1, 1, 1, 1, 1, 1]


In [30]:
distancia = sad(matrix)[0]
distancia

4

In [31]:
min_sad(matrix,2,[5,7],distancia,0)

4 4
4
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
Distancia aparente fuerte minima:  4
